In [2]:
!pip install -U "huggingface_hub"

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 534.5/534.5 kB 20.0 MB/s eta 0:00:00
  Attempting uninstall: huggingface_hub
    Found existing installation: huggingface-hub 0.36.0
    Uninstalling huggingface-hub-0.36.0:
      Successfully uninstalled huggingface-hub-0.36.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
transformers 4.57.3 requires huggingface-hub<1.0,>=0.34.0, but you have huggingface-hub 1.3.2 which is incompatible.


In [1]:
!hf auth login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To log in, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? [y/N]: y
Token is valid (permission: read).
The token `ReadToken1` has been saved to /root/.cache/huggingface/stored_tokens
Cannot authenticate through git-credential as no helper is defined on your machine.
You might have to re-authenticate whe

In [145]:
import pandas as pd

# Login using e.g. `huggingface-cli login` to access this dataset
df = pd.read_parquet("hf://datasets/mitulshah/transaction-categorization/default/train/0000.parquet")

In [146]:
df.head()

,transaction_description,category,country,currency
0,Wage,Income,USA,USD
1,Arby's (Contactless),Food & Dining,AUSTRALIA,AUD
2,Occupational Therapy,Healthcare & Medical,USA,USD
3,Potbelly Store Branch,Food & Dining,UK,GBP
4,Amazon - AUSTRALIA,Shopping & Retail,AUSTRALIA,AUD


In [147]:
df.isnull().sum()

,0
transaction_description,0
category,0
country,0
currency,0


In [148]:
df_required = df.drop(["country", "currency"], axis=1)

In [149]:
df_required.head(5)

,transaction_description,category
0,Wage,Income
1,Arby's (Contactless),Food & Dining
2,Occupational Therapy,Healthcare & Medical
3,Potbelly Store Branch,Food & Dining
4,Amazon - AUSTRALIA,Shopping & Retail


In [150]:
df_required.rename(columns={"transaction_description": "purpose_text", "category": "transaction_type"}, inplace=True)

In [151]:
df_required.head(5)

,purpose_text,transaction_type
0,Wage,Income
1,Arby's (Contactless),Food & Dining
2,Occupational Therapy,Healthcare & Medical
3,Potbelly Store Branch,Food & Dining
4,Amazon - AUSTRALIA,Shopping & Retail


In [152]:
transaction_types = df_required['transaction_type'].unique()

print("Labels:", transaction_types)
print("Number of labels:", len(transaction_types))

Labels: ['Income' 'Food & Dining' 'Healthcare & Medical' 'Shopping & Retail'
 'Utilities & Services' 'Entertainment & Recreation' 'Financial Services'
 'Charity & Donations' 'Transportation' 'Government & Legal']
Number of labels: 10


# Text preprocessing

In [153]:
df_required = df_required[:4000]

### Step1 - lowercase the text

In [154]:
df_required["purpose_text"] = df_required["purpose_text"].str.lower()

### Step2 - remove unnecessary symbols


In [155]:
import re

# 1. r'\(.*?\)' : Matches anything inside (brackets)
# 2. r'online'  : Matches the literal word "online"
# 3. r'[\',!\?#\-&@%\^]' : Matches any single character in this set


pattern = r'\(.*?\)|online|[\',!\?#\-&@%\^]'

df_required['purpose_text'] = df_required['purpose_text'].str.replace(pattern, '', regex=True)

# 4. r'\s+' : Remove whitespace when there are two or more

df_required['purpose_text'] = df_required['purpose_text'].str.replace(r'\s+', ' ', regex=True).str.strip()


### Step3. Let's remove numbers from the string, which are not income label.

In [156]:
pattern = r'[0-9]+'

df_required.loc[df_required['transaction_type'] != 'Income', 'purpose_text'] = df_required.loc[df_required['transaction_type'] != 'Income', 'purpose_text'].str.replace(pattern, '', regex=True)

In [157]:
df_required.head(10)

,purpose_text,transaction_type
0,wage,Income
1,arbys,Food & Dining
2,occupational therapy,Healthcare & Medical
3,potbelly store branch,Food & Dining
4,amazon australia,Shopping & Retail
5,dairy queen,Food & Dining
6,ucla medical uk center txn,Healthcare & Medical
7,walgreens,Healthcare & Medical
8,starlink,Utilities & Services
9,cox,Utilities & Services


In [158]:
df_required.to_csv("findata_processed.csv", index=False)